# Tarea 2

- Usando las funciones anteriores, o desarrollando las propias, escriba una función que pueda valorizar, por simulación de Montecarlo, calls y puts sobre bonos cero cupón con el modelo de HW. Considere puts y calls de plazo máximo igual a 1Y.
- Su función debe poder recibir la semilla.
- Compare sus resultados con las fórmulas del modelo y muestre que diferencias obtiene usando 1000 simulaciones y una semilla predeterminada.
- Valorice también por simulación de Montecarlo el bono cupón cero a 1Y. Muestre que diferencias obtiene respecto al valor de mercado del bono usando 1000 simulaciones y una semilla predeterminada.

In [1]:
from finrisk import QC_Financial_3 as Qcf
from typing import List, Tuple, Callable
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

# from modules
import auxiliary as aux

from numpy import random as rnd
import plotly.express as px
import pandas as pd
import numpy as np
import math
import scipy.stats

## Base de datos
Primero se captura la base de datos y se les hacen sus cambios correspondientes para realizar la tarea:
- Se anualiza los plazos.
- Se interpolan los valores de las tazas.


In [2]:
curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [3]:
curva

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929
5,61,0.000781,0.999870
6,92,0.000811,0.999796
7,125,0.000781,0.999733
8,152,0.000760,0.999683
9,182,0.000750,0.999626


In [4]:
# math.exp(-curva.tasa.iloc[32]*curva.t.iloc[32])

In [5]:
curva['t'] = curva['plazo'] / 365.0

In [6]:
zcurva = interp1d(curva['t'],
                  curva['tasa'],
                  kind='cubic',
                  fill_value="extrapolate")

## Monte Carlo


### Funciones complementarias
En esta sección se definen las funciones que ayudarán a la función principal, la cual valorizará la opción.


In [7]:


def zrate(t: float) -> float:
    return zcurva(t)

def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta

def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

def theta(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

def b_hw(gamma: float, t: float, T: float) -> float:
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose = False):
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    #if verbose:
    #    print("c1: " + str(c1))
    #    print("c2: " + str(c2))
    #    print("c3: " + str(c3))
    return c1 + c2 - c3

def zero_hw(r: float,
            gamma: float,
            sigma: float,
            zrate: float,
            fwd: float,
            t: float,
            T: float) -> float:
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

def option(e: float, Spot: float, strike: float) -> float:
    
    return max(e*Spot - e*strike, .0000)



### Función principal
La siguiente función corresponde al enunciado de la tarea, la cual simula un bono zero cupón con n simulaciones y T períodos, les resta a cada uno el valor del Strike, lo lleva a valor presente y calcula su promedio. Dicha función arrojando el valor presente y el valor del bono zero cupón.


In [8]:
### Funciones complementarias

def sim_hw_many(e, Spot, Strikes, gamma, sigma, theta, t, T, r0, num_sim, num_steps, seed = None):
    """
    """
    dt = 1 / 264.0
    num_steps += 1
    
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(0, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta(i * dt)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
            
    # Se toman los ultimos retornos        
    last_rates = [sim[:][-1] for sim in sim]
    
    # Se calcula los precios de los bonos cero cupon para cada simulacion
    precios = [zero_hw(r, gamma, sigma, zrate, fwd, t, T) * Spot for r in last_rates]
    
    valor_zero = np.average(precios)
    
    # Son los payoff e*(s-k) para cada simulacion
    payoffs = {Strike:[option(e, Spot, Strike) for Spot in precios] for Strike in Strikes}
   
    
    # "vp" comentado pues es una forma de calcular el valor presente
    #vp = [math.exp(-1.0 * float(zrate(T-t))) * np.average(payoffs[Strike]) for Strike in Strikes]
    
    # Los siguientes codigos no dependen de "valor zero" ni de "payoffs"
    
    
    #Se calcula el factor de descuto simulado para cada Strike
    df=0
    for sim in sim:
        df += math.exp(-dt * np.sum(sim))
    ez= df / num_sim 
    
    # "vp_sim" toma el valor presente de cada opcion simulacion como exp*(-r*T)*e*(S-K) para cada simulacion
    #vp_sim={Strike:[math.exp(-1.0 * float(zrate(T-t)*(T-t))) * option(e, Spot, Strike) for Spot in precios] for Strike in Strikes}
    vp_sim={Strike:[ez * option(e, Spot, Strike) for Spot in precios] for Strike in Strikes}
    
    # "vp" calcula el promedio de las opciones simuladas que fueron llevados a valor presente
    vp=[ np.average(vp_sim[Strike]) for Strike in Strikes]
    
    #se retorna el valor presente de la opción por Strike y tambie se retorna el valor del bono cero cupon.
    return vp, valor_zero

### Ejemplo Monte Carlo
Se evalúa el valor presente de la opción y el valor del bono bajo los siguientes parámetros. Cabe mencionar que la función acepta varios valores de Strike como matriz.


In [9]:
# IMPORTANTE: El imput Strike debe estar con corchete, la idea es obtener distintos valores presentes con distintos Strikes
# para realizar comparaciones entre simulaciones.

e = -1
Spot = 100
Strikes=[98, 99, 100, 101, 102]
#Strikes = [100]
gamma = .5
sigma = .015
t = 0
T = 1
r0 = zrate(0)
num_sim = 1000
num_steps = 263
seed = 1234

vp, valor_zero = sim_hw_many(
    e, Spot, Strikes, gamma, sigma, theta, t, T, r0, num_sim, num_steps, seed)

print(vp)
print(valor_zero)
math.exp(-zrate(T) * T)

[0.003981428179522913, 0.06313310999804062, 0.3728235609382379, 1.0874897296122767, 2.035094773005987]
99.96674281217366


0.9992978688293875

## Fórmula cerrada de Hull-White


### Funciones complementarias
Nuevamente se definen las funciones complementarias pero esta vez es para la fórmula cerrada.


In [10]:
def Sz(gamma, sigma, To, Tb, r0) -> float:
    aux = math.sqrt((sigma**2) / (2 * gamma) * (1 - math.exp(-2 * gamma * To)))
    return b_hw(gamma, 0, To)*aux


def d1(Spot, Strikes, gamma, sigma, To, Tb, r0) -> float:
    dfTb = Spot * math.exp(-zrate(Tb) * Tb)
    dfTo = Strikes * math.exp(-zrate(To) * To)
    c1 = math.log(dfTb / dfTo)
    c2 = (Sz(gamma, sigma, To, Tb, r0) ** 2) / 2
    #c3 = math.sqrt(Sz(gamma, sigma, To, Tb, r0))
    c3 = Sz(gamma, sigma, To, Tb, r0)
    return (c1 + c2) / c3


def d2(Spot, Strikes, gamma, sigma, To, Tb, r0) -> float:
    return d1(Spot, Strikes, gamma, sigma, To, Tb, r0) - Sz(gamma, sigma, To, Tb, r0)

### Funciones complementarias
En esta parte se define la función cerrada de Hull-White propiamente tal, la cual se asemeja bastante a la de Black-Scholes.


In [11]:
def option_hw(e, Spot, Strikes, gamma, sigma, To, Tb, r0):
    nd1 = scipy.stats.norm.cdf(e*d1(Spot, Strikes, gamma, sigma, To, Tb, r0))
    nd2 = scipy.stats.norm.cdf(e*d2(Spot, Strikes, gamma, sigma, To, Tb, r0))
    dfTb = math.exp(-zrate(Tb) * Tb)
    dfTo = math.exp(-zrate(To) * To)
    NSpot = Spot * dfTb / dfTo
    c1 = e*NSpot*nd1
    c2 = e*Strikes*nd2
    return math.exp(-zrate(To) * To)*(c1-c2)

### Ejemplo Hull-White


In [36]:
e = 1
Spot = 100
Strike = 98
gamma = .5
sigma = .015
To = 1
Tb = 2
r0 = zrate(0)
vp2 = option_hw(e, Spot, Strike, gamma, sigma, To, Tb, r0)
vp2

1.9612536434544676

In [37]:
import hull_white as hw

In [38]:
hw.zcb_call_put(
    hw.CallPut.CALL,
    strike=Strike/100,
    to=To,
    tb=Tb,
    r0=r0,
    zo=math.exp(-zrate(To) * To),
    zb=math.exp(-zrate(Tb) * Tb),
    gamma=gamma,
    sigma=sigma) * 100

1.9612536434544858

In [39]:
e = -1
Spot = 100
Strike = 102
gamma = .5
sigma = .015
To = 1
Tb = 2
r0 = zrate(0)
vp2 = option_hw(e, Spot, Strike, gamma, sigma, To, Tb, r0)
vp2

2.04707229189534

In [40]:
hw.zcb_call_put(
    hw.CallPut.PUT,
    strike=Strike/100,
    to=To,
    tb=Tb,
    r0=r0,
    zo=math.exp(-zrate(To) * To),
    zb=math.exp(-zrate(Tb) * Tb),
    gamma=gamma,
    sigma=sigma) * 100

2.0470722918953355

## Comparaciones de valorización
Luego de tener las funciones de Monte Carlo y la de Hull-white, se compararán los resultados de cada una bajo distintos parámetros.

En esta parte demora algo en compilarlo todo pues se hacen varias simulaciones.


Primero se evalúa con valores promedios o comunes en los parámetros $\gamma$ y $\sigma$


In [13]:
# Gamma y sigma con valores intermedios

resultados = []

e = 1
Spot = 100
Strikes = [98, 99, 100, 101, 102]
gamma = .5
sigma = .2
t = 0
T = 1
To = 1
Tb = 1
r0 = zrate(0)
num_sim = 1000
num_steps = 264
seed = 2000
vp, valor_zero = sim_hw_many(
    e, Spot, Strikes, gamma, sigma, theta, t, T, r0, num_sim, num_steps, seed)

for i in range(len(Strikes)):
    resultados.append((Strikes[i], vp[i], option_hw(
        e, Spot, Strikes[i], gamma, sigma, To, Tb, r0)))
df_resultados = pd.DataFrame(resultados, columns=['K', 'MC', 'HW'])
print(df_resultados)

     K        MC        HW
0   98  6.028021  5.998858
1   99  5.527626  5.476007
2  100  5.055627  4.985315
3  101  4.613787  4.526404
4  102  4.199539  4.098705


In [14]:
for i in range(len(Strikes)):
    print(
        f'Para un valor de strike {df_resultados.K.iloc[i]}, Monte carlo y Hull White estimaron :{df_resultados.MC.iloc[i]: .4f} | {df_resultados.HW.iloc[i]: .4f} ')

Para un valor de strike 98, Monte carlo y Hull White estimaron : 6.0280 |  5.9989 
Para un valor de strike 99, Monte carlo y Hull White estimaron : 5.5276 |  5.4760 
Para un valor de strike 100, Monte carlo y Hull White estimaron : 5.0556 |  4.9853 
Para un valor de strike 101, Monte carlo y Hull White estimaron : 4.6138 |  4.5264 
Para un valor de strike 102, Monte carlo y Hull White estimaron : 4.1995 |  4.0987 


Con gamma y sigma en valores intermedios, tal parece que la función de Monte Carlo sobrestima ligeramente las opciones con respecto a la formula cerrada de hull White, sin embargo, ambos son bastante parecidos así que esta diferencia se le puede atribuir a la aleatoriedad.


Ahora se evalúa $\gamma$ con un valor alto y bajo


In [15]:
# Gamma alto

resultados = []

e = 1
Spot = 100
Strikes = [98, 99, 100, 101, 102]
gamma = .9
sigma = .2
t = 0
T = 1
To = 1
Tb = 1
r0 = zrate(0)
num_sim = 1000
num_steps = 264
seed = 2000
vp, valor_zero = sim_hw_many(
    e, Spot, Strikes, gamma, sigma, theta, t, T, r0, num_sim, num_steps, seed)

for i in range(len(Strikes)):
    resultados.append((Strikes[i], vp[i], option_hw(
        e, Spot, Strikes[i], gamma, sigma, To, Tb, r0)))
df_resultados = pd.DataFrame(resultados, columns=['K', 'MC', 'HW'])
print(df_resultados)

     K        MC        HW
0   98  4.558811  4.631596
1   99  4.041480  4.082885
2  100  3.566786  3.578867
3  101  3.133248  3.119154
4  102  2.732192  2.702825


In [16]:
for i in range(len(Strikes)):
    print(
        f'Para un valor de strike {df_resultados.K.iloc[i]}, Monte carlo y Hull White estimaron :{df_resultados.MC.iloc[i]: .4f} | {df_resultados.HW.iloc[i]: .4f} ')

Para un valor de strike 98, Monte carlo y Hull White estimaron : 4.5588 |  4.6316 
Para un valor de strike 99, Monte carlo y Hull White estimaron : 4.0415 |  4.0829 
Para un valor de strike 100, Monte carlo y Hull White estimaron : 3.5668 |  3.5789 
Para un valor de strike 101, Monte carlo y Hull White estimaron : 3.1332 |  3.1192 
Para un valor de strike 102, Monte carlo y Hull White estimaron : 2.7322 |  2.7028 


In [17]:
# Gamma bajo

resultados = []

e = 1
Spot = 100
Strikes = [98, 99, 100, 101, 102]
gamma = .9
sigma = .2
t = 0
T = 1
To = 1
Tb = 1
r0 = zrate(0)
num_sim = 1000
num_steps = 264
seed = 2000
vp, valor_zero = sim_hw_many(
    e, Spot, Strikes, gamma, sigma, theta, t, T, r0, num_sim, num_steps, seed)

for i in range(len(Strikes)):
    resultados.append((Strikes[i], vp[i], option_hw(
        e, Spot, Strikes[i], gamma, sigma, To, Tb, r0)))
df_resultados = pd.DataFrame(resultados, columns=['K', 'MC', 'HW'])
print(df_resultados)

     K        MC        HW
0   98  4.558811  4.631596
1   99  4.041480  4.082885
2  100  3.566786  3.578867
3  101  3.133248  3.119154
4  102  2.732192  2.702825


In [18]:
for i in range(len(Strikes)):
    print(f'Para un valor de strike {df_resultados.K.iloc[i]}, Monte carlo y Hull White estimaron :{df_resultados.MC.iloc[i]: .4f} | {df_resultados.HW.iloc[i]: .4f} ')


Para un valor de strike 98, Monte carlo y Hull White estimaron : 4.5588 |  4.6316 
Para un valor de strike 99, Monte carlo y Hull White estimaron : 4.0415 |  4.0829 
Para un valor de strike 100, Monte carlo y Hull White estimaron : 3.5668 |  3.5789 
Para un valor de strike 101, Monte carlo y Hull White estimaron : 3.1332 |  3.1192 
Para un valor de strike 102, Monte carlo y Hull White estimaron : 2.7322 |  2.7028 


Ya sea con un $\gamma$ alto o bajo, ambos modelos son bastante parecidos. **AD:** "bastante parecidos" no es una argumentación precisa, hay que mostrar (al menos) error absoluto y/o relativo.

Ahora se va a cambiar el $\sigma$ con valores altos y bajos.


In [19]:
# sigma alto

resultados = []

e = 1
Spot = 100
Strikes = [98, 99, 100, 101, 102]
gamma = .5
sigma = .9
t = 0
T = 1
To = 1
Tb = 1
r0 = zrate(0)
num_sim = 1000
num_steps = 264
seed = 2000
vp, valor_zero = sim_hw_many(
    e, Spot, Strikes, gamma, sigma, theta, t, T, r0, num_sim, num_steps, seed)

for i in range(len(Strikes)):
    resultados.append((Strikes[i], vp[i], option_hw(
        e, Spot, Strikes[i], gamma, sigma, To, Tb, r0)))
df_resultados = pd.DataFrame(resultados, columns=['K', 'MC', 'HW'])
print(df_resultados)

     K         MC         HW
0   98  22.591897  22.946956
1   99  22.206658  22.547758
2  100  21.825614  22.155467
3  101  21.447334  21.769981
4  102  21.070142  21.391197


In [20]:
for i in range(len(Strikes)):
    print(f'Para un valor de strike {df_resultados.K.iloc[i]}, Monte carlo y Hull White estimaron :{df_resultados.MC.iloc[i]: .4f} | {df_resultados.HW.iloc[i]: .4f} ')


Para un valor de strike 98, Monte carlo y Hull White estimaron : 22.5919 |  22.9470 
Para un valor de strike 99, Monte carlo y Hull White estimaron : 22.2067 |  22.5478 
Para un valor de strike 100, Monte carlo y Hull White estimaron : 21.8256 |  22.1555 
Para un valor de strike 101, Monte carlo y Hull White estimaron : 21.4473 |  21.7700 
Para un valor de strike 102, Monte carlo y Hull White estimaron : 21.0701 |  21.3912 


In [21]:
# sigma bajo

resultados = []

e = 1
Spot = 100
Strikes = [98, 99, 100, 101, 102]
gamma = .5
sigma = .015
t = 0
T = 1
To = 1
Tb = 1
r0 = zrate(0)
num_sim = 1000
num_steps = 264
seed = 2000
vp, valor_zero = sim_hw_many(
    e, Spot, Strikes, gamma, sigma, theta, t, T, r0, num_sim, num_steps, seed)

for i in range(len(Strikes)):
    resultados.append((Strikes[i], vp[i], option_hw(
        e, Spot, Strikes[i], gamma, sigma, To, Tb, r0)))
df_resultados = pd.DataFrame(resultados, columns=['K', 'MC', 'HW'])
print(df_resultados)

     K        MC        HW
0   98  1.928434  2.003782
1   99  1.013277  1.067100
2  100  0.357089  0.374141
3  101  0.066050  0.069923
4  102  0.008895  0.005959


In [22]:
for i in range(len(Strikes)):
    print(f'Para un valor de strike {df_resultados.K.iloc[i]}, Monte carlo y Hull White estimaron :{df_resultados.MC.iloc[i]: .4f} | {df_resultados.HW.iloc[i]: .4f} ')


Para un valor de strike 98, Monte carlo y Hull White estimaron : 1.9284 |  2.0038 
Para un valor de strike 99, Monte carlo y Hull White estimaron : 1.0133 |  1.0671 
Para un valor de strike 100, Monte carlo y Hull White estimaron : 0.3571 |  0.3741 
Para un valor de strike 101, Monte carlo y Hull White estimaron : 0.0660 |  0.0699 
Para un valor de strike 102, Monte carlo y Hull White estimaron : 0.0089 |  0.0060 


Tanto con una valor de $\sigma$ alto o bajo, los modelos son bastante cercanos entre sí, así que ambos modelos conversan con un $\sigma$ cualquiera.

Ahora se evalúa la sensibilidad de ambos modelos variando el Strike.

In [23]:
# Strikes extremos

resultados = []

e = 1
Spot = 100
Strikes = [1, 50, 100, 150, 300]
gamma = .5
sigma = .2
t = 0
T = 1
To = 1
Tb = 1
r0 = zrate(0)
num_sim = 1000
num_steps = 264
seed = 2000
vp, valor_zero = sim_hw_many(
    e, Spot, Strikes, gamma, sigma, theta, t, T, r0, num_sim, num_steps, seed)

for i in range(len(Strikes)):
    resultados.append((Strikes[i], vp[i], option_hw(
        e, Spot, Strikes[i], gamma, sigma, To, Tb, r0)))
df_resultados = pd.DataFrame(resultados, columns=['K', 'MC', 'HW'])
print(df_resultados)

     K         MC            HW
0    1  98.587806  9.893049e+01
1   50  49.582871  4.996489e+01
2  100   5.055627  4.985315e+00
3  150   0.015619  2.437318e-03
4  300   0.000000  1.970807e-18


In [24]:
for i in range(len(Strikes)):
    print(f'Para un valor de strike {df_resultados.K.iloc[i]}, Monte carlo y Hull White estimaron :{df_resultados.MC.iloc[i]: .4f} | {df_resultados.HW.iloc[i]: .4f} ')


Para un valor de strike 1, Monte carlo y Hull White estimaron : 98.5878 |  98.9305 
Para un valor de strike 50, Monte carlo y Hull White estimaron : 49.5829 |  49.9649 
Para un valor de strike 100, Monte carlo y Hull White estimaron : 5.0556 |  4.9853 
Para un valor de strike 150, Monte carlo y Hull White estimaron : 0.0156 |  0.0024 
Para un valor de strike 300, Monte carlo y Hull White estimaron : 0.0000 |  0.0000 


Por lo anterior, se puede ver que ambos modelos logran asemejarse con distintos valores extremos del Strike, por lo que se puede decir que ambos modelos estan bien estructurados y que conversan entre ellos independientemente de sus parámetros.

Ahora se va a realizar el último punto, donde se compara el valor del bono zero cupón versus el valor del mercado (zrate), con los parámetros clásicos de las clases, es decir, $\sigma$=0.5 y $\sigma$=0.015

In [25]:
e = 1
Spot = 100
Strikes = [98, 99, 100, 101, 102]
gamma = .5
sigma = .015
t = 0
T = 1
To = 1
Tb = 1
r0 = zrate(0)
num_sim = 1000
num_steps = 264
seed = 1234
vp, valor_zero = sim_hw_many(
    e, Spot, Strikes, gamma, sigma, theta, t, T, r0, num_sim, num_steps, seed)

print(
    f'Valor de un bono cupon cero por Monte carlo y por el mercado :{valor_zero: .4f} | {zero_hw(zrate(1), gamma, sigma, zrate, fwd, t, T) * Spot: .4f} ')
print('La función de Monte Carlo lo subestima muy ligeramente, pero tanto la funcion de Monte Carlo como el indice del Mercado resultaron ser bastante parecidos')

Valor de un bono cupon cero por Monte carlo y por el mercado : 99.9602 |  99.9365 
La función de Monte Carlo lo subestima muy ligeramente, pero tanto la funcion de Monte Carlo como el indice del Mercado resultaron ser bastante parecidos


**AD:** Muy claro. Bien que compararon resultados cambiando parámetros. No se evalúan las puts, no hay cálculo de error.